In [1]:
import sys
sys.path.append('../')

from axolotl.model.transformer import SEDD
import torch
import torch.nn as nn

config = {
    'model': {
        'name': 'small',
        'type': 'ddit',
        'hidden_size': 768,
        'cond_dim': 128,
        'length': 4096,
        'n_blocks': 12,
        'n_heads': 12,
        'scale_by_sigma': True,
        'dropout': 0.1,
        'label_dropout': 0.1, # for classifier-free guidance (cfg)
    },
    
    'graph': {
        'type': 'absorb',
    },
    'ngpus': 1,
    'tokens': 23, # 23 for uniform
    'num_labels': 2, # prokaryotic (0), eukaryotic (1)
}

/home/kkj/axolotl/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from peft import get_peft_model, LoraConfig, TaskType


model = SEDD(config).to('cuda')

peft_config = LoraConfig(
    target_modules=["qkv_proj", "fc", "out_proj"], # TODO? , 'mlp', timestep embedding, positional embedding, class embedding
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
)

# for module in model.named_modules():
#     print(module)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

# for module in peft_model.named_modules():
#     print(module)

trainable params: 448,704 || all params: 92,885,848 || trainable%: 0.4831
